# Contexte
Le gouvernement péruvien vient d’autoriser les aventuriers en quête de trésors à explorer les 85 182
km² du département de la Madre de Dios. Vous devez réaliser un système permettant de suivre les
déplacements et les collectes de trésors effectuées par les aventuriers. Le gouvernement péruvien
étant très à cheval sur les bonnes pratiques de code, il est important de réaliser un code de qualité,
lisible, et maintenable (oui, ça veut dire avec des tests) !

In [57]:
import re
import numpy as np
#from numpy import zeros,array

In [58]:
nbAdventurers = 0
tabAdventurers = []    # np.empty((2,5), list)
print(tabAdventurers)
gameMap = None

[]


## Lire le fichier d'entrée

In [59]:
def readFile(inputPath, outputPath):
    inputFile = open(inputPath, 'r')
    outputFile = open(outputPath, 'w')
    outputStrAcommeA = "# {A comme Aventurier} - {Nom de l’aventurier} - {Axe horizontal} - {Axe vertical} - {Orientation} - {Nb. trésors ramassés} \n"
    global nbAdventurers
    global tabAdventurers
    nbCellsWidth = 0
    nbCellsHeight = 0
    global gameMap
    xMontain = None
    yMontain = None
    xTreasure = None
    yTreasure = None
    nbTreasures = None
    
    
    while True:
        # Get next line from file
        line = inputFile.readline()
        if not line:
            print("break while true")
            break
        
        #print(line)
        if re.search("^(A|a).*-", line) == None:
            if re.search("^(#).*{(A|a).*comme", line):
                outputFile.write(outputStrAcommeA)
            else:
                # Carte
                if re.search("^(C|c).*-.*\d", line):
                    nbCellsWidth = int(re.findall("\d+", line)[0])
                    nbCellsHeight = int(re.findall("\d+", line)[1])
                    gameMap = np.empty((nbCellsHeight,nbCellsWidth), list)
                # Montagne    
                if re.search("^(M|m).*-.*\d", line):
                    xMontain = int(re.findall("\d+", line)[0])
                    yMontain = int(re.findall("\d+", line)[1])
                    gameMap[yMontain][xMontain] = "M"
                # Trésor
                if re.search("^(T|t).*-.*\d", line):
                    xTreasure = int(re.findall("\d+", line)[0])
                    print("x treasure : " + str(xTreasure))
                    yTreasure = int(re.findall("\d+", line)[1])
                    print("y treasure : " + str(yTreasure))
                    nbTreasures = int(re.findall("\d+", line)[2])
                    print("nb treasure : " + str(nbTreasures))
                    gameMap[yTreasure][xTreasure] = ["T", nbTreasures]
                                          
                print("write line.")
                print(line)
                outputFile.write(line)
            
            
        else:
            # Adventurer(s)
            nbAdventurers += 1
            listElems = []
            listElems.append("A")
            nameAdv = re.findall("[a-zA-Z]{2,}", line)[0]
            listElems.append(nameAdv)
            columnAdv = int(re.findall("\d+", line)[0])
            listElems.append(columnAdv)
            lineAdv = int(re.findall("\d+", line)[1])
            listElems.append(lineAdv)
            orientationAdv = re.findall("\s+(N|n|S|s|E|e|O|o){1}\s+", line)[0]
            listElems.append(orientationAdv)
            motionSequence = re.findall("[a-zA-Z]{2,}", line)[1]
            listElems.append(motionSequence)
            nbTreasuresFound = 0
            listElems.append(nbTreasuresFound)
            #.append(columnAdv).append(lineAdv).append(orientationAdv).append(motionSequence)
            print(listElems)
            indice = nbAdventurers-1
            #print("indice : " + str(indice))
            #tabAdventurers[indice] = listElems  #.append(columnAdv).append(lineAdv).append(orientationAdv).append(motionSequence)
            tabAdventurers.append(listElems)
            print(tabAdventurers[0])
            #timTab = np.append(timTab, np.array([[1,2,3,4,5]]), axis=0)
            #tabAdventurers = np.append(tabAdventurers, np.array([listElems]), axis=0)
            gameMap[lineAdv][columnAdv] = listElems
                        
            print("output character")
            txtAname = str(re.findall("^A|a.*-.*-", line)) #+ "{{coordonnées}} {{nb trésors trouvés}}"
            outputFile.write(txtAname)
            
            #print("# nb adventurers : " + str(nbAdventurers))
            #print("name : " + str(nameAdv))
            #print("column number : " + str(columnAdv))
            #print("line number : " + str(lineAdv))
            #print("orientation : "+str(orientationAdv))
            #print("motion sequence :"+str(motionSequence))
               
    for item in tabAdventurers:
        print("item :: " + str(item))            
    inputFile.close()
    outputFile.close()
    print("all files closed")
    
    print("gameMap")
    print(".....................")
    print(gameMap)

In [60]:
cheminEntree = './ressources/input02.carte'
cheminSortie = './ressources/output02.carte'

In [61]:
readFile(cheminEntree, cheminSortie)

write line.
# {C comme Carte} - {Nb. de case en largeur} - {Nb. de case en hauteur}

write line.
C - 3 - 4

write line.
# {M comme Montagne} - {Axe horizontal} - {Axe vertical}

write line.
M - 0 - 2

['A', 'Indiana', 1, 1, 'S', 'AADADA', 0]
['A', 'Indiana', 1, 1, 'S', 'AADADA', 0]
output character
break while true
item :: ['A', 'Indiana', 1, 1, 'S', 'AADADA', 0]
all files closed
gameMap
.....................
[[None None None]
 [None list(['A', 'Indiana', 1, 1, 'S', 'AADADA', 0]) None]
 ['M' None None]
 [None None None]]


In [62]:
print(tabAdventurers)
print("nb Adventurers :", nbAdventurers)
print(gameMap)

[['A', 'Indiana', 1, 1, 'S', 'AADADA', 0]]
nb Adventurers : 1
[[None None None]
 [None list(['A', 'Indiana', 1, 1, 'S', 'AADADA', 0]) None]
 ['M' None None]
 [None None None]]


In [68]:
def isInsideMap(y=int,x=int):
    gameMapHeight = gameMap.shape[0]
    gameMapWidth = gameMap.shape[1]
    
    if y < gameMapHeight and y >= 0 and x < gameMapWidth and x >= 0:
        return True
    else:
        return False

In [74]:
def getNewOrientation(orientation,nextMovement):
    if nextMovement != 'A':
        
        if nextMovement == 'D':
            if orientation == 'N':
                orientation = 'E'
            elif orientation == 'S':
                orientation = 'O'
            elif orientation == 'E':
                orientation = 'S'
            elif orientation == 'O':
                orientation = 'N'
        elif nextMovement == 'G':
            if orientation == 'N':
                orientation = '0'
            elif orientation == 'S':
                orientation = 'E'
            elif orientation == 'E':
                orientation = 'N'
            elif orientation == 'O':
                orientation = 'S'
            
    return orientation

In [77]:
def doSimulation(nbAdventurers, tabAdventurers, gameMap):
    gmHeight = gameMap.shape[0]
    gmWidth = gameMap.shape[1]
    print("Simulation...")
    
    while True:
        for i in range(len(tabAdventurers)):
            print(tabAdventurers[i])
            print("next move ")
            print(tabAdventurers[i][5])
            peopleName = tabAdventurers[i][1]
            coordX = int(tabAdventurers[i][2])
            coordY = int(tabAdventurers[i][3])
            orientation = tabAdventurers[i][4]
            movementsList = tabAdventurers[i][5]
            if movementsList:
                nextMovement = movementsList[0]
            else:
                 break
            print(peopleName)
            print(orientation)
            print(nextMovement)
            
            if len(movementsList) >= 1:
                
                # Fonction orientationChange(actualOrientation,movement)   'D' or 'G'
                orientation = getNewOrientation(orientation,nextMovement)
                   
        
                if orientation == 'S':
                    newCoordY = coordY + 1
                    # Cell is empty
                    # if (newCoordY < gmHeight)
                    if isInsideMap(newCoordY,coordX) and gameMap[newCoordY][coordX] == None:
                        gameMap[coordY][coordX] = None
                        tabAdventurers[i][3] = newCoordY
                        tabAdventurers[i][5] = tabAdventurers[i][5][1:]
                        gameMap[newCoordY][coordX] = tabAdventurers[i]
                    # Treasure in cell
                    if isInsideMap(newCoordY,coordX) and gameMap[newCoordY][coordX][0] == 'T': 
                        nbTreasures = int(gameMap[newCoordY][coordX][3])
                        if nbTreasures >= 1:
                            tabAdventurers[i][6] = tabAdventurers[i][6] + 1
                            gameMap[newCoordY][coordX][3] = int(gameMap[newCoordY][coordX][3]) - 1
                            
                                
                
            #for elm in range(len(tabAdventurers[i][5])):
            #tabAdventurers[i][5] = tabAdventurers[i][5][1:]   # [1:]
            #print(tabAdventurers[i][5])
        #i = 0
        break

In [78]:
doSimulation(nbAdventurers, tabAdventurers, gameMap)

Simulation...
['A', 'Indiana', 1, 3, 'S', 'DADA', 0]
next move 
DADA
Indiana
S
D


In [71]:
tabAdventurers

[['A', 'Indiana', 1, 2, 'S', 'ADADA', 0]]

In [72]:
gameMap

array([[None, None, None],
       [None, None, None],
       ['M', list(['A', 'Indiana', 1, 2, 'S', 'ADADA', 0]), None],
       [None, None, None]], dtype=object)